#  Codes du Projet du Cours IFT 7022 (Partie Application sur une interface graphique en utilisant Tkinter)

### Membre de l'équipe :

##### -Sekou Kaba
##### -Salah Eddine Khalil

# Importations des modules requis pour la réalisation de l'application (A installer au cas de besoin)

In [1]:
import os
import librosa
import IPython.display as ipd
from scipy.io import wavfile
from math import *
from tkinter import *
from tkinter import messagebox
from copy import deepcopy
import webbrowser as wb
from tkinter import filedialog
from keras.models import load_model
import numpy as np
import pandas as pd
import sounddevice as sd
import soundfile as sf
import csv as cssv
import warnings


warnings.filterwarnings("ignore")

Puis on defini un fonction de prédiction, qui prend en entré un audio, le réechantillone en 8000Hz Effectue par la suite la prédiction du label et retourne le label corespondant(Parmi les labels classes choisis)

In [2]:
def convertir(audio):
    classes=['down', 'go', 'left', 'no', 'off', 'on', 'right', 'stop', 'up', 'yes']
    model=load_model('best_model.hdf5')
    prob=model.predict(audio.reshape(1,8000,1))
    index=np.argmax(prob[0])
    return classes[index]


Nous deffinissons une fonction qui permet egalement d'enregistrer instantannement un audio en 1s et de le convertir

In [3]:
def ecouter():
    samplerate = 16000  
    duration = 1 #seconds
    filename = 'instantanee.wav'
    print("start")
    mydata = sd.rec(int(samplerate * duration), samplerate=samplerate,
        channels=1, blocking=True)
    print("end")
    sd.wait()
    sf.write(filename, mydata, samplerate)
    sample, sample_rate = librosa.load(filename, sr = 16000)
    sample = librosa.resample(sample, sample_rate, 8000)
    if(len(sample)== 8000) : 
        text=convertir(sample)
    else:
        text="impossible"
    ent1.delete('0.0','end')
    ent1.insert('0.0',text)
    os.system("rm "+filename)
    

A defaut d'ecouter l'utilisateur, nous deffinissons une fonction qui permet de charger un audio et de le convertir

In [4]:
def charger():
    global liste
    liste=filedialog.askdirectory()
    sons=os.listdir(liste)
    sons=[f for f in os.listdir(liste) if f.endswith('.wav')]
    global dsonss
    dsonss={}
    for son in sons:
        sample, sample_rate = librosa.load(liste+'/'+son, sr = 16000)
        sample = librosa.resample(sample, sample_rate, 8000)
        if(len(sample)== 8000) : 
            dsonss.update({son:convertir(sample)})
        else:
            dsonss.update({son:"impossible"})        
    ListF.delete(0, ListF.size())
    n=1
    
    ligne="ID Speech  \t \t                                  Text"
    for i,j in dsonss.items():
        ListF.insert(ListF.size() + 1, ligne)
        ligne = i + '  \t \t \t \t ' + j 
        n+=1
    ListF.insert(ListF.size() + 1, ligne)

Pour exporter les predictions dans un fichier csv

In [5]:
def export():
    try:
        with open(liste+'/Resultats.csv', 'w', newline='') as fichier:
            ecri = cssv.writer(fichier)
            ecri.writerow(['ID Speech', 'Text'])
            for i,j in dsonss.items():
                ecri.writerow([ i,j])    
        messagebox.showinfo("Succes","Exportés avec succes")
    except:
        messagebox.showerror("Erreur d'ouverture",
                     "Une erreur est survenue. Impossible d'exproter les données!")


Quelques fonctions auxiliaires, utilisées pour amelliorer l'interface :

In [7]:
#Pour actualiser l'interface

def refresh():
    b = messagebox.askyesnocancel('Actualiser ?', "Etes vous sure de vouloir rafraichir le Convertisseur?", )
    if b == True:
        ent1.delete('0.0','end')
        ListF.delete(0, ListF.size())

#Pour fermer l'interface

def detruire():
    b = messagebox.askyesnocancel('Quitter ?', "Etes vous sure de vouloir quitter le convertisseur?", )
    if b == True:
        converti.destroy()

#Un descrptif pdf de l'utilisation

def ouvr():
    try:
        open("converti.pdf",'r')
    except:
        messagebox.showinfo("!", "Veillez ouvrir manuellement le guide, présent dans le dossier de l'application!")

#Un descrptif video de l'utilisation        
def ouvr1():
    try:
         open("converti.mp4",'r')
    except:
         messagebox.showinfo("!", "Veillez ouvrir manuellement le guide, présent dans le dossier de l'application!")


Création d'une interface permettant la conversion de l'audio en text (Utilisant le module python)

In [8]:
converti=Tk()
converti.title('Projet Cours IFT-7022')
converti.geometry("1300x660")
converti.config(bg='plum4')
converti.resizable(width=False,height=False)

im1=PhotoImage(file='L1.png')
im2=PhotoImage(file='L2.png')
im3=PhotoImage(file='L3.png')
converti.iconphoto(False, im2)
lf1 = LabelFrame(converti,width=10, height=0, text="  Demarrer l'écoute ",font="Arial 16 italic", fg='black', padx=5, pady=20)
lf1.place(x=0,y=150)
lf2 = LabelFrame(converti,width=20, height=0, text="  Charger les fichiers  ",font="Arial 16 italic", fg='black', padx=5, pady=20)
lf2.place(x=734,y=10)
panel1 = Label(lf1, image=im1)
panel1.pack(side='left', expand=True)
panel2 = Label(lf2, image=im3)
panel2.pack(side='right', expand=True)
converti.lift()

ent1 = Text(converti,fg='black',bg='white', font=('Times', '30', 'italic'), width=34, height=1)
ent1.place(x=15, y=200)


B2=Button(converti,text="Rafraichir", bg='PaleTurquoise1', font=('Times', '20', 'bold italic'), fg='black',width=10,command=refresh)
B2.place(x=550,y=580)
B3=Button(converti,text="Ecouter & Convertir", bg='royal blue', font=('Times', '15', 'bold italic'), fg='black',width=16,command=ecouter)
B3.place(x=520,y=163)
B4=Button(converti,text="Charger & Convertir", bg='royal blue', font=('Times', '15', 'bold italic'), fg='black',width=16,command=charger)
B4.place(x=1100,y=23)
B5=Button(converti,text="Quitter", font=('Times', '20', 'bold italic'), fg='black',width=10,command=detruire)
B5.place(x=2,y=580)
DefilFy=Scrollbar(converti,orient='vertical',width=17)
DefilFy.place(x=1280, y=60)
DefilFx= Scrollbar(converti, orient='horizontal',width=17)
DefilFx.place(x=745, y=40)
ListF=Listbox(converti,bg='peach puff',fg='black',font=('Times','13','italic'), width=60, height=29,xscrollcommand=DefilFx.set,yscrollcommand=DefilFy.set)
ListF.place(x=745, y=63)
DefilFy.config(command= ListF.xview)
DefilFy.config(command= ListF.yview)


menub = Menu(converti)
converti['menu'] = menub
menub.config(bg="CadetBlue4", font=('Times', '15', 'bold italic'), fg='black', relief='groove')
m1 = Menu(menub, bg='PaleTurquoise1', font=('Times', '12', 'bold italic'), fg='black')
m2 = Menu(menub, bg='PaleTurquoise1', font=('Times', '12', 'bold italic'), fg='black')
menub.add_cascade(label='Fichier', menu=m1)
menub.add_cascade(label='Aides', menu=m2)
m1.add_command(label="Exporter sous format csv", command=export)
m1.add_command(label="Rafraichir la Page", command=refresh)
m1.add_command(label="Quitter le convertisseur", command=detruire)
m1.config(tearoff=0)
m2.config(tearoff=0)

s1m2 = Menu(m2)
s2m2 = Menu(m2)
s3m2 = Menu(m2)
m2.add_command(label="Guide d'utilisation", command=ouvr)
m2.add_command(label="Manuel Video", command=ouvr1)
m2.add_command(label="Pour plus veillez nous contacter")


converti.mainloop()

start
end
1/1 [==============================] - 0s 127ms/step
start
end
1/1 [==============================] - 0s 124ms/step
start
end
1/1 [==============================] - 0s 123ms/step
